In [ ]:
import numpy as np
import pandas as pd 
import geopandas as gpd
import os
from shapely.geometry import Point
from shapely.geometry import MultiPolygon, Polygon
import time
import psycopg2
from io import StringIO
import matplotlib.pyplot as plt
from shapely import wkb

In [ ]:
def connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user, db_password):
    start_time = time.time()
    
    conn = None
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            database=database_name,
            user=db_user,
            password=db_password,
            host=postgres_host,
            port=postgres_port
        )
        print("PostgreSQL connection opened")
        
        # Create a cursor object
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(query)
        print("Query executed successfully")
        
        # Fetch all rows from the executed query
        rows = cursor.fetchall()
        
        # Get column names from the cursor
        colnames = [desc[0] for desc in cursor.description]
        
        # Convert the result to a pandas DataFrame
        df = pd.DataFrame(rows, columns=colnames)
        
        return df

    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

    finally:
        # Closing the connection
        if conn:
            cursor.close()
            conn.close()
            print("PostgreSQL connection closed.")

    print(f'connect_to_postgres took {time.time() - start_time} seconds to complete')

In [ ]:
postgres_host = 'localhost'
postgres_port = 5432 
database_name = 'busyness'
db_user = 'crafty'
db_password = 'winner'
table_name = 'public.busyness_data'

In [ ]:
query = '''
    SELECT
    DATE_TRUNC('minute', TO_TIMESTAMP(timestamp)) AS minute, SUM(busynessscore) AS total_busyness
    FROM
        busyness_data
    WHERE 
        zonenumber = 70 and mode != 'subway'
    GROUP BY
        DATE_TRUNC('minute', TO_TIMESTAMP(timestamp))
    ORDER BY
        minute; 
    '''

In [ ]:
query = '''
    SELECT
    DATE_TRUNC('hour', TIMESTAMP WITH TIME ZONE 'epoch' + timestamp * INTERVAL '1 SECOND') +
        ((EXTRACT(MINUTE FROM TIMESTAMP WITH TIME ZONE 'epoch' + timestamp * INTERVAL '1 SECOND')::int / 10) * 10 || ' minutes')::interval AS ten_minute_interval,
    SUM(busynessscore) AS total_busyness
FROM
    busyness_data
WHERE 
    zonenumber = 70 AND mode != 'subway'
GROUP BY
    DATE_TRUNC('hour', TIMESTAMP WITH TIME ZONE 'epoch' + timestamp * INTERVAL '1 SECOND') +
        ((EXTRACT(MINUTE FROM TIMESTAMP WITH TIME ZONE 'epoch' + timestamp * INTERVAL '1 SECOND')::int / 10) * 10 || ' minutes')::interval
ORDER BY
    ten_minute_interval; 
    '''



In [ ]:
df = connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user, db_password)
df

In [ ]:
df = df.drop(df.index[:3])
df

In [ ]:
# df['unix_timestamp'] = df['minute'].apply(lambda x: x.timestamp()/10**9)

# Plotting the data
plt.figure(figsize=(12, 6))
plt.plot(df['unix_timestamp'], df['total_busyness'], linestyle='-')
plt.title('Busyness Score Over Time')
plt.xlabel('Unix Timestamp')
plt.ylabel('Busyness Score')
plt.grid(True)
plt.tight_layout()
# plt.xlim(1.6444,1.9)
plt.xlim(1.696*10**9,1.7*10**9)
plt.show()

In [ ]:
# df['unix_timestamp'] = df['minute'].apply(lambda x: x.timestamp()/10**9)

plt.figure(figsize=(12, 6))
plt.plot(df['ten_minute_interval'], df['cumulative_busyness'], linestyle='-')
plt.title('Busyness Score Over Time')
plt.xlabel('ten_minute_interval')
plt.ylabel('Busyness Score')
plt.grid(True)
plt.tight_layout()
plt.xlim(1.67,1.68)
# plt.xlim(1.68475,1.6850)
plt.show()

In [ ]:
# df['ten_minute_interval'] = pd.to_datetime(df['minute'])
df['ten_minute_interval'] = pd.to_datetime(df['ten_minute_interval'], utc=True)
df = df.sort_values(by='ten_minute_interval')

# Extract date from 'minute' column for grouping
df['date'] = df['ten_minute_interval'].dt.date

# Compute cumulative sum of 'total_busyness' within each day
df['cumulative_busyness'] = df.groupby('date')['total_busyness'].cumsum()

# Drop the 'date' column as it's no longer needed
df = df.drop(columns=['date'])


df['unix_timestamp'] = df['ten_minute_interval'].apply(lambda x: time.mktime(x.timetuple()))
df

In [ ]:
# Sort DataFrame by 'minute' (if not already sorted)
df = df.sort_values(by='minute')

# Split into training and testing sets
train_size = int(len(df) * 0.1)  # 80% for training
train_data = df.iloc[:train_size]
test_data = df.iloc[:train_size]

In [ ]:
train_data

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA model
model = ARIMA(train_data['cumulative_busyness'][23000:], order=(2, 0, 0))  # Example: AR(1) model
model_fit = model.fit()

# Print model summary (optional)
print(model_fit.summary())

In [ ]:
# Make predictions on test set
predictions = model_fit.forecast(len(test_data))

# Evaluate predictions against actual values
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(test_data['cumulative_busyness'], predictions)
print(f'Mean Squared Error (MSE): {mse}')

In [ ]:
forecast = model_fit.forecast(steps=10)
print(forecast)

In [ ]:
predictions = model_fit.forecast(len(test_data))  # Example: Replace with your predictions


In [ ]:
predictions = model_fit.forecast(len(test_data))  # Example: Replace with your predictions


import matplotlib.pyplot as plt

# Plotting actual data
plt.figure(figsize=(14, 7))
# plt.plot(test_data['minute'][10:], test_data['cumulative_busyness'][10:], label='Actual')

# Plotting predictions
plt.plot(test_data['minute'][10:], predictions[10:], label='Predicted')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Total Busyness')
plt.title('Actual vs Predicted Total Busyness')
plt.legend()

# Display plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(train_data['minute'][10:], train_data['cumulative_busyness'][10:])
# plt.xlim("2002-10-25 17:20:00+00:00",)

In [ ]:
train_data

In [ ]:
train_data = train_data.drop(train_data.index[:3])

In [ ]:
train_data.to_csv('test_business_score_data.csv')

In [ ]:
train_data['minute'] = pd.to_datetime(train_data['minute'])

# Add Unix timestamp column
train_data['unix_timestamp'] = train_data['minute'].apply(lambda x: time.mktime(x.timetuple()))

In [ ]:
# train_data
start_day = 1648480740.0-432000
end_day = 1648480740.0

In [ ]:
max_timestamp = train_data['unix_timestamp'].max()
print(max_timestamp)

In [ ]:
start_day = 1.696*10**9
end_day = 1.7*10**9

# Create subset DataFrame with Unix timestamps between start_day and end_day
subset_df = df[(df['unix_timestamp'] >= start_day) & (df['unix_timestamp'] <= end_day)]


In [ ]:
subset_df

In [ ]:
print(min(y),max(y))

In [ ]:
x = subset_df['unix_timestamp']
y = subset_df['total_busyness']

plt.plot(x,y)

In [ ]:
subset_df.info()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA model
model = ARIMA(subset_df['cumulative_busyness'], order=(5, 1, 10))  # Example: AR(1) model
model_fit = model.fit()

In [ ]:
predictions = model_fit.forecast(len(subset_df))  # Example: Replace with your predictions


In [ ]:
import matplotlib.pyplot as plt

# Plotting actual data
plt.figure(figsize=(14, 7))
plt.plot(subset_df['unix_timestamp'], subset_df['cumulative_busyness'], label='Actual')

# Plotting predictions
plt.plot(subset_df['unix_timestamp'], predictions, label='Actual')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Total Busyness')
plt.title('Actual vs Predicted Total Busyness')
plt.legend()

# Display plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

In [ ]:
from datetime import datetime
import numpy as np             #for numerical computations like log,exp,sqrt etc
import pandas as pd            #for reading & storing data, pre-processing
import matplotlib.pylab as plt #for visualization
#for making sure matplotlib plots are generated in Jupyter notebook itself
%matplotlib inline             
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10, 6

In [ ]:
# subset_df = subset_df.drop(columns = {'minute','total_busyness'})
# new_order = ['unix_timestamp','cumulative_busyness']
subset_df = subset_df[new_order]

In [ ]:
df = df.drop(df.index[:3])
df

In [ ]:
#Determine rolling statistics
rolmean = subset_df['total_busyness'].rolling(window=24).mean() #window size 12 denotes 12 months, giving rolling mean at yearly level
rolstd = subset_df['total_busyness'].rolling(window=24).std()
print(rolmean,rolstd)

In [ ]:
orig = plt.plot(subset_df['unix_timestamp'],subset_df['total_busyness'], color='blue', label='Original')
mean = plt.plot(subset_df['unix_timestamp'],rolmean, color='red', label='Rolling Mean')
std = plt.plot(subset_df['unix_timestamp'],rolstd, color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
df

In [ ]:
print('Results of Dickey Fuller Test:')
dftest = adfuller(subset_df['total_busyness'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
dfoutput

In [ ]:
indexedDataset_logScale = np.log(subset_df['total_busyness']-min(subset_df['total_busyness'])+1)
print(subset_df['total_busyness']-min(subset_df['total_busyness'])+1)

In [ ]:
indexedDataset_logScale = np.log(subset_df['total_busyness']-min(subset_df['total_busyness'])+1)
plt.plot(subset_df['unix_timestamp'],indexedDataset_logScale)#subset_df['total_busyness'])

In [ ]:
movingAverage = indexedDataset_logScale.rolling(window=24).mean()
movingSTD = indexedDataset_logScale.rolling(window=24).std()
plt.plot(indexedDataset_logScale)
plt.plot(movingAverage, color='red')

In [ ]:
datasetLogScaleMinusMovingAverage = indexedDataset_logScale - movingAverage
datasetLogScaleMinusMovingAverage.head(12)

#Remove NAN values
datasetLogScaleMinusMovingAverage.dropna(inplace=True)
datasetLogScaleMinusMovingAverage.head(10)

In [ ]:
def test_stationarity(timeseries,windows):
    
    #Determine rolling statistics
    movingAverage = timeseries.rolling(window=windows).mean()
    movingSTD = timeseries.rolling(window=windows).std()
    
    #Plot rolling statistics
    orig = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(movingAverage, color='red', label='Rolling Mean')
    std = plt.plot(movingSTD, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey–Fuller test:
    print('Results of Dickey Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(datasetLogScaleMinusMovingAverage)

In [ ]:
datasetLogDiffShifting = indexedDataset_logScale - indexedDataset_logScale.shift()
plt.plot(datasetLogDiffShifting)

In [ ]:
#AR Model
#making order=(2,1,0) gives RSS=1.5023
model = ARIMA(indexedDataset_logScale, order=(2,1,0))
results_AR = model.fit()
plt.plot(datasetLogDiffShifting)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_AR.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting AR model')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA


In [ ]:
df

In [ ]:
model = ARIMA(indexedDataset_logScale, order=(0,1,2))
results_MA = model.fit()
plt.plot(datasetLogDiffShifting)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting MA model')

In [ ]:
df

In [ ]:
df

In [ ]:
print(query)

In [ ]:
df = connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user, db_password)

# df['unix_timestamp'] = df['ten_minute_interval'].apply(lambda x: x.timestamp()/10**9)
# df = df.drop(columns = {'ten_minute_interval'})

df

In [ ]:
df['pos_bus'] = df['cumulative_busyness']-min(df['cumulative_busyness'])+1
df['pos_bus'] = df['pos_bus'].astype(float)

In [ ]:
df['Rolling_5'] = df['total_busyness'].rolling(window=5).mean()
df['Rolling_10'] = df['total_busyness'].rolling(window=10).mean()
df['Rolling_20'] = df['total_busyness'].rolling(window=20).mean()
df['Rolling_50'] = df['total_busyness'].rolling(window=50).mean()
df['Rolling_100'] = df['total_busyness'].rolling(window=100).mean()
df['Rolling_200'] = df['total_busyness'].rolling(window=200).mean()
df['Rolling_500'] = df['total_busyness'].rolling(window=500).mean()
df['Rolling_1000'] = df['total_busyness'].rolling(window=1000).mean()

In [ ]:
df['cumulative_busyness'] = df['cumulative_busyness'].astype(float)
df['ts'] = df['cumulative_busyness']-movingAverage

df

In [ ]:
# indexedDataset_logScale = np.log(df['pos_bus'])
movingAverage = df['cumulative_busyness'].rolling(window=24).mean()

In [ ]:
x = df['hour']
y =df['pos_bus']

plt.plot(df['ts'][17000:17500])
plt.xlim(17000,17500)
plt.show()
test_stationarity(df['ts'][17000:17500])

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Fit ARIMA model
model = ARIMA(df['ts'][17000:17500], order=(2, 0, 0))  # Example: AR(1) model
model_fit = model.fit()

# Print model summary (optional)
print(model_fit.summary())

In [ ]:
predictions = model_fit.forecast(len(df['ts'][17000:17500]))  # Example: Replace with your predictions


import matplotlib.pyplot as plt

# Plotting actual data
plt.figure(figsize=(14, 7))
plt.plot(df['ts'][17000:17500], label='Actual')

# Plotting predictions
plt.plot(predictions, label='Predicted')

# Adding labels and title
plt.xlabel('Time')
plt.ylabel('Total Busyness')
plt.title('Actual vs Predicted Total Busyness')
plt.legend()

# Display plot
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()


In [ ]:
model = ARIMA(df['ts'][17000:17500], order=(0,1,2))
results_MA = model.fit()
plt.plot(df['ts'][17000:17500],'bo')
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting MA model')

In [ ]:
area: zone
    
time: prediction

In [ ]:
x = df['unix_timestamp']

 

y =df['Rolling_5'] 
plt.plot(x,y)
plt.show()


y =df['Rolling_10'] 

plt.plot(x,y)
plt.show()

y =df['Rolling_20'] 
plt.plot(x,y)
plt.show()


y =df['Rolling_50'] 

plt.plot(x,y)
plt.show()

y =df['Rolling_100']

plt.plot(x,y)
plt.show()

y =df['Rolling_200']

plt.plot(x,y)
plt.show()

y =df['Rolling_500']

plt.plot(x,y)
plt.show()

y =df['Rolling_1000']

plt.plot(x,y)
plt.xlim(1.696,1.698)
plt.show()


In [ ]:
plt.plot(x,y)
# plt.xlim(1.696,1.698)
plt.show()

In [ ]:
query = '''
WITH min_max_timestamps AS (
    SELECT
        MIN(TO_TIMESTAMP(timestamp)) AS min_timestamp,
        MAX(TO_TIMESTAMP(timestamp)) AS max_timestamp
    FROM
        busyness_data
    WHERE 
        zonenumber = 70
        AND mode != 'subway'
),

hour_series AS (
    SELECT generate_series(
        DATE_TRUNC('hour', min_max_timestamps.min_timestamp),
        DATE_TRUNC('hour', min_max_timestamps.max_timestamp),
        '1 hour'::interval
    ) AS hour
    FROM min_max_timestamps
)

SELECT
    hour_series.hour AS hour,
    COALESCE(SUM(busynessscore), 0) AS total_busyness
FROM
    hour_series
LEFT JOIN (
    SELECT
        DATE_TRUNC('hour', TO_TIMESTAMP(timestamp)) AS hour,
        SUM(busynessscore) AS busynessscore
    FROM
        busyness_data
    WHERE 
        zonenumber = 70
        AND mode != 'subway'
    GROUP BY
        DATE_TRUNC('hour', TO_TIMESTAMP(timestamp))
) AS aggregated_data ON hour_series.hour = aggregated_data.hour
GROUP BY
    hour_series.hour
ORDER BY
    hour_series.hour;
'''

In [ ]:
df

In [ ]:
# indexedDataset_logScale = np.log(indexedDataset)
# df['ts_pos'] = df['ts'] - min(df['ts'])
df['logscale'] = np.log(df['ts_pos'])

In [ ]:
df

In [ ]:
plt.plot(df['logscale'])

In [ ]:
min(df['ts'])

In [ ]:
df['logscale+'] = df['logscale+'].fillna(0)

In [ ]:
test_stationarity(df['logscale+'],240)

In [ ]:
rolmean = df['logscale'].rolling(window=240).mean() #window size 12 denotes 12 months, giving rolling mean at yearly level
rolstd = df['logscale'].rolling(window=240).std()
print(rolmean,rolstd)

In [ ]:
orig = plt.plot(df['logscale'], color='blue', label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [ ]:
df['logscale+'] = df['logscale']-rolmean

In [ ]:
plt.plot(df['logscale+'])

In [ ]:
model = ARIMA(df['logscale+'], order=(2,1,2))
results_MA = model.fit()
plt.plot(df['logscale+'],'bo')
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting MA model')

In [ ]:
plt.plot(df['logscale+'],'bo')
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
plt.xlim(15000,17500)
print('Plotting MA model')

In [ ]:
#ACF & PACF plots

lag_acf = acf(df['logscale+'], nlags=20)
lag_pacf = pacf(df['logscale+'], nlags=20, method='ols')

#Plot ACF:
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')            

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')
            
plt.tight_layout()            

In [ ]:
df

In [ ]:
forecast = results_MA.forecast(steps=1000)

In [ ]:
# Forecast future values
n_steps = 10  # Number of future steps to forecast
forecast = results_MA.forecast(steps=n_steps)

# Create a DataFrame for the forecast results
forecast_index = pd.date_range(start=df.index[-1], periods=n_steps+1)
forecast_df = pd.DataFrame(forecast, index=forecast_index, columns=['forecast'])

# Plot the historical data, fitted values, and forecast
plt.figure(figsize=(10, 6))
plt.plot(df['logscale+'], 'bo-', label='Historical Data')
plt.plot(results_MA.fittedvalues, 'r--', label='Fitted Values')
plt.plot(forecast_df, 'go-', label='Forecast')

# Plot confidence intervals
plt.fill_between(forecast_index, conf_int[:, 0], conf_int[:, 1], color='gray', alpha=0.3)

plt.title('ARIMA Model Forecast')
plt.xlabel('Date')
plt.ylabel('Log Scale')
plt.legend()
plt.show()

# Print the forecast
print('Forecasted values:', forecast)

In [ ]:
forecast = results_MA.forecast(steps=1000)

plt.plot(df['logscale+'])
plt.plot(forecast)
plt.xlim(17000,18700)

In [ ]:
cumulative = df['total_busyness'].cumsum()
cumulative = cumulative.astype('float')

rolling_avg = cumulative.rolling(window = 24).mean()
rolling_avg.fillna(0, inplace=True)

cum_avg = cumulative - rolling_avg
possitive_shift = min(cum_avg)
possitive_cumulative = cum_avg - possitive_shift +1

# plt.plot(logscale)

# print(possitive_shift)
logscale = np.log(possitive_cumulative)
log2 = logscale - logscale.shift()

log2.dropna(inplace=True)
test_stationarity(log2,24)


In [ ]:
df

In [ ]:
# df['rolling'] = df['total_busyness'].rolling(window = 24).mean()
df['rolling'] = df['rolling'].fillna(0)

In [ ]:
plt.plot(df['rolling'])
plt.xlim(15000,17500)

In [ ]:
#Perform Augmented Dickey–Fuller test:
print('Results of Dickey Fuller Test:')
dftest = adfuller(df['rolling'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
print(dfoutput)

In [ ]:
#ACF & PACF plots

lag_acf = acf(log2, nlags=20)
lag_pacf = pacf(log2, nlags=20, method='ols')

#Plot ACF:
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Autocorrelation Function')            

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.title('Partial Autocorrelation Function')
            
plt.tight_layout()            

In [ ]:
model = ARIMA(log2, order=(6,0,0))
results_MA = model.fit()
plt.plot(log2,'b--')
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting MA model')

In [ ]:
forecast = results_MA.forecast(steps=1000)

plt.plot(log2)
plt.plot(forecast)
plt.xlim(17000,18700)

In [ ]:
model = ARIMA(log2, order=(2,1,2))
results_MA = model.fit()

plt.plot(log2,'b--')
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting MA model')
# plt.xlim(17000,17500)
# plt.ylim(-1,1)

In [ ]:
# Split into training and testing sets
train_size = int(len(log2) * 0.8)  # 80% for training
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

In [ ]:
print(len(test_data),len(train_data))

In [ ]:
# Split into training and testing sets
train_size = int(len(log2) * 0.8)  # 80% for training
train_data = log2.iloc[:train_size]
test_data = log2.iloc[train_size:]

model = ARIMA(train_data, order=(2,1,2))
results_MA = model.fit()

plt.plot(train_data,'b--')
plt.plot(results_MA.fittedvalues, color='red')
# plt.title('RSS: %.4f'%sum((results_MA.fittedvalues - datasetLogDiffShifting)**2))
print('Plotting MA model')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Assume log2 contains the time series data
# Split the data into training and testing sets
train_size = int(len(log2) * 0.8)  # 80% for training
train_data = df['ts_pos'].iloc[:train_size]
test_data = df['ts_pos'].iloc[train_size:]

# Train the ARIMA model on the training set
model = ARIMA(train_data, order=(2,1,2))
results_MA = model.fit()

# Make predictions on the test set
predictions = results_MA.forecast(steps=len(test_data))

# Evaluate the model using mean squared error
mse = mean_squared_error(test_data, predictions)
print(f'Mean Squared Error: {mse:.4f}')

# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))
plt.plot(train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Test Data', color='green')
plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.title('ARIMA Model Evaluation')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(17000,17500)
# plt.ylim(-1,1)

plt.show()


In [ ]:
# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))
plt.plot(train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Test Data', color='green')
plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.title('ARIMA Model Evaluation')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(17000,17500)
plt.show()


In [ ]:
test_stationarity(df['ts_pos'],24)

In [ ]:
df

In [ ]:
df['cumulative_busyness'] = df['total_busyness'].cumsum()
df['moving_average'] = df['cumulative_busyness'].rolling(window = 24)
df['ts'] = df['cumulative_busyness'] - df['moving_average']

# ts = df['total_busyness'].cumsum() - df['total_busyness'].cumsum().rolling(window = 24)

In [ ]:
ts = df['total_busyness'].astype('float').cumsum() - (df['total_busyness'].astype('float').cumsum()).rolling(window = 24).mean()

In [ ]:
ts

In [ ]:
def ts(dataframe,col):
    data = dataframe[col].astype('float')
    cumsum = data.cumsum()
    moving_avg = cumsum.rolling(window = 24).mean()
    ts = cumsum - moving_avg
    return ts
# df['total_busyness'].cumsum() - (df['total_busyness'].cumsum()).rolling(window = 24)


In [ ]:
ts(df,col = 'total_busyness')

In [ ]:
pipeline = df.pipe(ts, col='total_busyness')
 
# calling pipeline
pipeline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error

# Assume log2 contains the time series data
# Split the data into training and testing sets
train_size = int(len(log2) * 0.8)  # 80% for training
train_data = pipeline.iloc[:train_size]
test_data = pipeline.iloc[train_size:]

# Train the ARIMA model on the training set
model = ARIMA(train_data, order=(2,1,2))
results_MA = model.fit()

# Make predictions on the test set
predictions = results_MA.forecast(steps=len(test_data))

# Evaluate the model using mean squared error
mse = mean_squared_error(test_data, predictions)
print(f'Mean Squared Error: {mse:.4f}')

# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))
plt.plot(train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Test Data', color='green')
plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.title('ARIMA Model Evaluation')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(17000,17500)
# plt.ylim(-1,1)

plt.show()

In [ ]:
# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))
plt.plot(train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Test Data', color='green')
plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.title('ARIMA Model Evaluation')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(15500,16000)
# plt.ylim(-1,1)

plt.show()

In [ ]:
def query(x):  
    query = f'''
    WITH min_max_timestamps AS (
        SELECT
            MIN(TO_TIMESTAMP(timestamp)) AS min_timestamp,
            MAX(TO_TIMESTAMP(timestamp)) AS max_timestamp
        FROM
            busyness_data
        WHERE 
            zonenumber = {x}
            AND mode != 'subway'
    ),

    hour_series AS (
        SELECT generate_series(
            DATE_TRUNC('hour', min_max_timestamps.min_timestamp),
            DATE_TRUNC('hour', min_max_timestamps.max_timestamp),
            '1 hour'::interval
        ) AS hour
        FROM min_max_timestamps
    )

    SELECT
        hour_series.hour AS hour,
        COALESCE(SUM(busynessscore), 0) AS total_busyness
    FROM
        hour_series
    LEFT JOIN (
        SELECT
            DATE_TRUNC('hour', TO_TIMESTAMP(timestamp)) AS hour,
            SUM(busynessscore) AS busynessscore
        FROM
            busyness_data
        WHERE 
            zonenumber = {x}
            AND mode = 'taxi'
        GROUP BY
            DATE_TRUNC('hour', TO_TIMESTAMP(timestamp))
    ) AS aggregated_data ON hour_series.hour = aggregated_data.hour
    GROUP BY
        hour_series.hour
    ORDER BY
        hour_series.hour;
    '''
    return query

In [ ]:
def query(x):  
    query = f'''
    WITH min_max_timestamps AS (
        SELECT
            MIN(TO_TIMESTAMP(timestamp)) AS min_timestamp,
            MAX(TO_TIMESTAMP(timestamp)) AS max_timestamp
        FROM
            busyness_data_2
        WHERE 
            zonenumber = {x}
    ),

    hour_series AS (
        SELECT generate_series(
            DATE_TRUNC('hour', min_max_timestamps.min_timestamp),
            DATE_TRUNC('hour', min_max_timestamps.max_timestamp),
            '1 hour'::interval
        ) AS hour
        FROM min_max_timestamps
    )

    SELECT
        hour_series.hour AS hour,
        COALESCE(SUM(busynessscore), 0) AS total_busyness
    FROM
        hour_series
    LEFT JOIN (
        SELECT
            DATE_TRUNC('hour', TO_TIMESTAMP(timestamp)) AS hour,
            SUM(busynessscore) AS busynessscore
        FROM
            busyness_data_2
        WHERE 
            zonenumber = {x}
        GROUP BY
            DATE_TRUNC('hour', TO_TIMESTAMP(timestamp))
    ) AS aggregated_data ON hour_series.hour = aggregated_data.hour
    GROUP BY
        hour_series.hour
    ORDER BY
        hour_series.hour;
    '''
    return query

In [ ]:
query(14)

In [ ]:
df = connect_to_postgres(query(x), postgres_host, postgres_port, database_name, db_user, db_password)

# df['unix_timestamp'] = df['ten_minute_interval'].apply(lambda x: x.timestamp()/10**9)
# df = df.drop(columns = {'ten_minute_interval'})

df

In [ ]:
for i in range(1,3):
    df = connect_to_postgres(query(i), postgres_host, postgres_port, database_name, db_user, db_password)
    pipeline = df.pipe(ts, col='total_busyness')

    # calling pipeline
    # pipeline

    train_size = int(len(log2) * 0.8)  # 80% for training
    train_data = pipeline.iloc[:train_size]
    test_data = pipeline.iloc[train_size:]

    # Train the ARIMA model on the training set
    model = ARIMA(train_data, order=(2,1,2))
    results_MA = model.fit()

    # Make predictions on the test set
    predictions = results_MA.forecast(steps=len(test_data))

    # Evaluate the model using mean squared error
    mse = mean_squared_error(test_data, predictions)
    print(f'Mean Squared Error: {mse:.4f}')

    # Plot the training data, test data, and predictions
    plt.figure(figsize=(12, 6))
    plt.plot(train_data, label='Training Data', color='blue')
    plt.plot(test_data.index, test_data, label='Test Data', color='green')
    plt.plot(test_data.index, predictions, label='Predictions', color='red')
    plt.legend()
    plt.title(f'ARIMA Model Evaluation plot{i}')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.xlim(17000,17500)
#     plt.ylim(-1,1)

    plt.show()

In [ ]:
df = connect_to_postgres(query(1), postgres_host, postgres_port, database_name, db_user, db_password)

# df['unix_timestamp'] = df['ten_minute_interval'].apply(lambda x: x.timestamp()/10**9)
# df = df.drop(columns = {'ten_minute_interval'})

df

In [ ]:
pipeline = df.pipe(ts, col='total_busyness')

# calling pipeline
# pipeline

train_size = int(len(pipeline) * 0.8)  # 80% for training
train_data = pipeline.iloc[:train_size]
test_data = pipeline.iloc[train_size:]

# Train the ARIMA model on the training set
model = ARIMA(train_data, order=(1,1,2))
results_MA = model.fit()

# Make predictions on the test set
predictions = results_MA.forecast(steps=len(test_data))

# Evaluate the model using mean squared error
mse = mean_squared_error(test_data, predictions)
print(f'Mean Squared Error: {mse:.4f}')

# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))

plt.plot(train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Test Data', color='green')

plt.plot(results_MA.fittedvalues, label='Fit Data', color='red')


plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.title(f'ARIMA Model Evaluation plot{i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(17000,17500)
#     plt.ylim(-1,1)

plt.show()

In [ ]:
#ACF & PACF plots

lag_acf = acf(df['logshift'], nlags=50)
lag_pacf = pacf(df['logshift'], nlags=50, method='ols')

#Plot ACF:
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axvline(6)

plt.title('Autocorrelation Function')            

#Plot PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0, linestyle='--', color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle='--', color='gray')
plt.axvline(2)
plt.title('Partial Autocorrelation Function')
    
plt.tight_layout() 

In [ ]:
pipeline[24:]

In [ ]:
plt.plot(df['hour'][-200:],df['log'][-200:])

In [ ]:
df

In [ ]:
df['positive'] = df['total_busyness'] - min(df['total_busyness']) + 2
df['log'] = np.log(df['positive'].astype('float'))
rolling_mean = df['log'].rolling(window = 240).mean()
print(rolling_mean)
df['logshift'] = df['log'] - rolling_mean

In [ ]:
df['log'] = np.log(df['positive'].astype('float'))
rolling_mean = df['log'].rolling(window = 240).mean()
print(rolling_mean)
df['logshift'] = df['log'] - rolling_mean

In [ ]:
rolling_mean = df['log'].rolling(window = 240).mean()
print(rolling_mean)
df['logshift'] = df['log'] - rolling_mean

In [ ]:
df['logshift'] = df['logshift'].fillna(0)
# test_stationarity(df['logshift'])
test_stationarity(df['logshift'],240)

In [ ]:
# pipeline = df.pipe(ts, col='total_busyness')

# calling pipeline
# pipeline

train_size = int(len(df['logshift']) * 0.8)  # 80% for training
train_data = df['logshift'].iloc[:train_size]
test_data = df['logshift'].iloc[train_size:]

# Train the ARIMA model on the training set
model = ARIMA(train_data, order=(2,1,0))
results_MA = model.fit()

# Make predictions on the test set
predictions = results_MA.forecast(steps=len(test_data))

# Evaluate the model using mean squared error
mse = mean_squared_error(test_data, predictions)
print(f'Mean Squared Error: {mse:.4f}')

# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))

plt.plot(train_data, label='Training Data', color='blue')
plt.plot(test_data.index, test_data, label='Test Data', color='green')

plt.plot(results_MA.fittedvalues, label='Fit Data', color='red')


plt.plot(test_data.index, predictions, label='Predictions', color='red')
plt.legend()
plt.title(f'ARIMA Model Evaluation plot{i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(7400,7600)
#     plt.ylim(-1,1)

plt.show()

In [ ]:
plt.plot(df['logshift'])
plt.xlim(15000,15200)

In [ ]:
import numpy as np
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import RandomForestRegressor
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.split import temporal_train_test_split

y = df['logshift']
y_train, y_test = temporal_train_test_split(y)
fh = np.arange(1, len(y_test) + 1)  # forecasting horizon
regressor = RandomForestRegressor()
forecaster = make_reduction(
    regressor,
    strategy="recursive",
    window_length=24,
)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
smape = MeanAbsolutePercentageError()
smape(y_test, y_pred)
plt.plot(y_test)
plt.plot(y_pred)
plt.xlim(17000,17500)

In [ ]:
plt.plot(y_test)
plt.plot(y_pred)
plt.xlim(17000,17500)

In [ ]:
def time_series(df):
    minimum = min(df['total_busyness'])
    positive = df['total_busyness'] - minimum + 2
    log= np.log(positive.astype('float'))
    rolling_mean = log.rolling(window = 240).mean()
    logshift = log - rolling_mean
    logshift = logshift.fillna(0)
    
    return logshift,rolling_mean, minimum

In [ ]:
pipeline = df.pipe(time_series)
pipeline

In [ ]:
for i in range(1,10):
    df = connect_to_postgres(query(i), postgres_host, postgres_port, database_name, db_user, db_password)
    ts,roll_mean,minimum = df.pipe(time_series)

    # calling pipeline
    # pipeline


    # Train the ARIMA model on the training set
    y = ts
    y_train, y_test = temporal_train_test_split(y)
    fh = np.arange(1, len(y_test) + 1)  # forecasting horizon
    regressor = RandomForestRegressor()
    forecaster = make_reduction(
        regressor,
        strategy="recursive",
        window_length=24,
    )
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    smape = MeanAbsolutePercentageError()
    print('smape:',smape(y_test, y_pred))

    # Evaluate the model using mean squared error
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {mse:.4f}')

    # Plot the training data, test data, and predictions
    plt.figure(figsize=(12, 6))
    plt.plot(y_test, label='Test Data')
    plt.plot(y_pred, label='Predictions')
    plt.legend()
    plt.title(f'sktime Model Evaluation plot{i}')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.xlim(17000,17500)
#     plt.ylim(-1,1)

    plt.show()

In [ ]:
df = connect_to_postgres(query(1), postgres_host, postgres_port, database_name, db_user, db_password)

ts,roll_mean,minimum = df.pipe(time_series)

# calling pipeline
# pipeline

# Train the ARIMA model on the training set
y = ts
y_train, y_test = temporal_train_test_split(y)
fh = np.arange(1, len(y_test) + 1)  # forecasting horizon
regressor = RandomForestRegressor()
forecaster = make_reduction(
    regressor,
    strategy="recursive",
    window_length=24,
)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
smape = MeanAbsolutePercentageError()
print(smape(y_test, y_pred))

# Evaluate the model using mean squared error
# print(len(y_test),len(y_pred))
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.4f}')

# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Test Data')
plt.plot(y_pred, label='Predictions')
plt.legend()
plt.title(f'sktime Model Evaluation plot{i}')
plt.xlabel('Time')
plt.ylabel('Value')
plt.xlim(17000,17500)
#     plt.ylim(-1,1)

plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


# Calculate regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Print the regression metrics
print()
print(f'Mean Absolute Error (MAE): {mae:.4f}','\n')
print(f'Mean Squared Error (MSE): {mse:.4f}','\n')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}','\n')
print(f'R-squared (R²): {r2:.4f}','\n')


In [ ]:
print(len(test_data),len(y_pred))

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiPolygon
taxi_zone = gpd.read_file('NYCTaxiZones.geojson')
taxi_zone.set_geometry("geometry", inplace=True)


In [ ]:
def predict_func(lat,lng, timestamp,taxi_zone):

    point = Point(lng, lat)
    containing_polygon = taxi_zone['location_id'][taxi_zone['geometry'].contains(point)]
    return containing_polygon

In [ ]:
lng = 45
lat = -37
point = Point(lng, lat)

In [ ]:
predict_func(40.65,-74, 1,taxi_zone)

In [ ]:
taxi_zone

In [ ]:
df

In [ ]:
ts,roll_mean,minimum = df.pipe(time_series)

In [ ]:
minimum

In [ ]:
zone_list = taxi_zone['location_id'][taxi_zone['borough'] == 'Manhattan']

In [ ]:
taxi_zone_2 = taxi_zone[taxi_zone['location_id'].isin(zone_list)]
taxi_zone_2.to_csv('/Users/brianmcmahon/Documents/Research Practicum/Data/taxi_zone.csv')

In [ ]:
taxi_zone['location_id'] = taxi_zone['location_id'].astype('int')
taxi_zone.info()


In [ ]:
zone_list = [
    4, 24, 12, 13, 41, 45, 42, 43, 48, 50, 68, 79, 74, 75, 87, 88, 90, 125, 
    100, 107, 113, 114, 116, 120, 127, 128, 151, 140, 137, 141, 
    142, 152, 143, 144, 148, 158, 161, 162, 163, 164, 170, 166, 186, 
    209, 211, 224, 229, 230, 231, 239, 232, 233, 234, 236, 237, 238, 263, 
    243, 244, 246, 249, 261, 262
]
for i in zone_list:
    print(i)

In [ ]:
for i in zone_list:
    df = connect_to_postgres(query(i), postgres_host, postgres_port, database_name, db_user, db_password)
    ts,roll_mean,minimum = df.pipe(time_series)

    # calling pipeline
    # pipeline


    # Train the ARIMA model on the training set
    y = ts
    y_train, y_test = temporal_train_test_split(y)
    fh = np.arange(1, len(y_test) + 1)  # forecasting horizon
    regressor = RandomForestRegressor()
    forecaster = make_reduction(
        regressor,
        strategy="recursive",
        window_length=24,
    )
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    smape = MeanAbsolutePercentageError()
    print('smape:',smape(y_test, y_pred))

    # Evaluate the model using mean squared error
    # Calculate regression metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Print the regression metrics
    print()
    print(f'Mean Absolute Error (MAE): {mae:.4f}','\n')
    print(f'Mean Squared Error (MSE): {mse:.4f}','\n')
    print(f'Root Mean Squared Error (RMSE): {rmse:.4f}','\n')
    print(f'R-squared (R²): {r2:.4f}','\n')

    # Plot the training data, test data, and predictions
    plt.figure(figsize=(12, 6))
    plt.plot(y_test, label='Test Data')
    plt.plot(y_pred, label='Predictions')
    plt.legend()
    plt.title(f'sktime Model Evaluation plot{i}')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.xlim(17000,17500)
#     plt.ylim(-1,1)

    plt.show()

In [ ]:
def busyness_force(lat,lng,tazi_zone,list_of_zones):
    

In [ ]:
def distance(lat_1,lng_1,lat_2,lng_2):
    '''
    Returns the radial distance between two points
    '''
    x = lng_1-lng_2
    y = lat_1-lat_2
    h = np.sqrt(x**2 + y**2)
    
    return h

In [ ]:
taxi_zone

In [ ]:
taxi_zone['centroid'] = taxi_zone['geometry'].centroid


In [ ]:
df

In [ ]:
df = connect_to_postgres(query(262), postgres_host, postgres_port, database_name, db_user, db_password)

ts,roll_mean,minimum = df.pipe(time_series)

# calling pipeline
# pipeline

# Train the ARIMA model on the training set
y = ts
y_train, y_test = temporal_train_test_split(y)
# fh = np.arange(1, len(y_test) + 1)  # forecasting horizon
fh = np.arange(1, 2160)  # forecasting horizon 3 months

regressor = RandomForestRegressor()
forecaster = make_reduction(
    regressor,
    strategy="recursive",
    window_length=168,
)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
# smape = MeanAbsolutePercentageError()
# print(smape(y_test, y_pred))

# # Evaluate the model using mean squared error
# # print(len(y_test),len(y_pred))
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error: {mse:.4f}')

# Plot the training data, test data, and predictions
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Test Data')
plt.plot(y_pred, label='Predictions')
plt.legend()
plt.title(f'sktime Model Evaluation plot{12}')
plt.xlabel('Time')
plt.ylabel('Value')
# plt.xlim(17000,17500)
#     plt.ylim(-1,1)

plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_train)
plt.plot(y_test, label='Test Data')
plt.plot(y_pred, label='Predictions')
plt.legend()
plt.title(f'sktime Model Evaluation plot{12}')
plt.xlabel('Time')
plt.ylabel('Value')
# plt.xlim(13500,14000)

# plt.xlim(16000,16500)
# plt.ylim(-1,1)

plt.show()

In [ ]:
df

In [ ]:
postgres_host = 'localhost'
postgres_port = 5432 
database_name = 'busyness'
db_user = 'crafty'
db_password = 'winner'
table_name = 'public.busyness_data_2'
df = connect_to_postgres(query(42), postgres_host, postgres_port, database_name, db_user, db_password)
df

In [ ]:
for i in zone_list:
    df = connect_to_postgres(query(i), postgres_host, postgres_port, database_name, db_user, db_password)
    mean_value = df['total_busyness'].mean().mean()
    ts,roll_mean,minimum = df.pipe(time_series)
    
    y = ts
    y_train, y_test = temporal_train_test_split(y)
#     fh = np.arange(1, len(y_test) + 1)  # forecasting horizon
    fh = np.arange(1, 2160)  # forecasting horizon 3 months

    regressor = RandomForestRegressor()
    forecaster = make_reduction(
        regressor,
        strategy="recursive",
        window_length=168,
    )
    forecaster.fit(y_train)
    y_pred = forecaster.predict(fh)
    smape = MeanAbsolutePercentageError()
    
    with open(f'/Users/brianmcmahon/Documents/Research Practicum/Models/forecaster_model_zone_{i}.pkl', 'wb') as file:
        pickle.dump(forecaster, file)
    
    
    print('smape:',smape(y_test, y_pred[:len(y_test)]))
    smape_2 = smape(y_test, y_pred[:len(y_test)])
    # Evaluate the model using mean squared error
    # Calculate regression metrics
    mae = mean_absolute_error(y_test, y_pred[:len(y_test)])
    mse = mean_squared_error(y_test, y_pred[:len(y_test)])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred[:len(y_test)])

    # Print the regression metrics
    print()
    print(f'Mean Absolute Error (MAE): {mae:.4f}','\n')
    print(f'Mean Squared Error (MSE): {mse:.4f}','\n')
    print(f'Root Mean Squared Error (RMSE): {rmse:.4f}','\n')
    print(f'R-squared (R²): {r2:.4f}','\n')
    
    stats_file_path = f'/Users/brianmcmahon/Documents/Research Practicum/Models/model_statistics_zone_{i}.txt'
    write_model_statistics(stats_file_path, i, smape_2, mae, mse, rmse, r2, roll_mean, minimum, mean_value)
    
    start_hour = max(df['hour'])  # specify the start hour
    zone = i  # specify the zone
    x = np.exp(y_pred)*mean_value
    new_df = create_dataframe(start_hour, zone, x)
    
    postgres_host = 'localhost'
    postgres_port = 5432 
    database_name = 'busyness'
    db_user = 'crafty'
    db_password = 'winner'
    table_name = 'public.busyness_score'

    upload_to_postgres(new_df, postgres_host, postgres_port, database_name, db_user, db_password, table_name)
    
    # Plot the training data, test data, and predictions
    plt.figure(figsize=(12, 6))
    plt.plot(y_test, label='Test Data')
    plt.plot(y_pred, label='Predictions')
    plt.legend()
    plt.title(f'sktime Model Evaluation plot{i}')
    plt.xlabel('Time')
    plt.ylabel('Value')
#     plt.xlim(17000,17500)
#     plt.ylim(-1,1)

    plt.show()

In [ ]:
import pickle


In [ ]:
# Save the model to a file
with open('/Users/brianmcmahon/Documents/Research Practicum/Models/forecaster_model.pkl', 'wb') as file:
    pickle.dump(forecaster, file)

In [ ]:
def write_model_statistics(file_path, zone, smape, mae, mse, rmse, r2, roll_mean, minimum,mean_value):
    with open(file_path, 'w') as f:
        f.write(f"Model Statistics for Zone: {zone}\n")
        f.write(f"sMAPE: {smape:.4f}\n")
        f.write(f"Mean Absolute Error (MAE): {mae:.4f}\n")
        f.write(f"Mean Squared Error (MSE): {mse:.4f}\n")
        f.write(f"Root Mean Squared Error (RMSE): {rmse:.4f}\n")
        f.write(f"R-squared (R²): {r2:.4f}\n")
#         f.write(f"Rolling Mean: {roll_mean}\n")
        f.write(f"Minimum: {minimum}\n")
        f.write(f"Mean Value of DataFrame: {mean_value:.4f}\n")
    print(f"Statistics written to {file_path}")

In [ ]:
df['total_busyness'].mean()

In [ ]:
df

In [ ]:
import os
import pandas as pd

# Function to compute the mean of a DataFrame and update the statistics file
def update_statistics_with_mean(stats_file_path, df):
    # Compute the mean value of the DataFrame
    mean_value = df['total_busyness'].mean().mean()

    # Read the existing statistics text file
    with open(stats_file_path, 'r') as file:
        lines = file.readlines()

    # Add the mean value to the statistics
    lines.append(f"Mean Value of DataFrame: {mean_value:.4f}\n")

    # Write the updated statistics back to the text file
    with open(stats_file_path, 'w') as file:
        file.writelines(lines)
    
    print(f"Updated {stats_file_path} with mean value.")

# Directory where the statistics files are stored
stats_directory = '/Users/brianmcmahon/Documents/Research Practicum/Models'

# Iterate over each zone and update the corresponding statistics file
for i in zone_list:
    # Load the DataFrame for the current zone
    df = connect_to_postgres(query(i), postgres_host, postgres_port, database_name, db_user, db_password)

    # Path to the statistics file
    stats_file_path = os.path.join(stats_directory, f'model_statistics_zone_{i}.txt')

    # Update the statistics file with the mean value of the DataFrame
    update_statistics_with_mean(stats_file_path, df)


In [ ]:
plt.plot(y_pred)

In [ ]:
plt.plot(np.exp(y_pred)*14.309122203098108)

In [ ]:
max(df['hour'])

In [ ]:
max(df['hour'])+datetime.month(2)

In [ ]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta


In [ ]:
predict_until = max(df['hour']) + relativedelta(months=2)


In [ ]:
fh = np.arange(1, 2160)  # forecasting horizon 3 months


In [ ]:
3*30*24

In [ ]:
x = np.exp(y_pred)*14.309122203098108

In [ ]:
x

In [ ]:
def create_dataframe(start_hour, zone,x):
    # Starting datetime
#     start_time = datetime.strptime(start_hour, "%Y-%m-%d %H:%M:%S")
    
    # Create a list of hours iterating one hour for each row
    hours = [start_hour + timedelta(hours=i) for i in range(len(x))]
    
    # Create the DataFrame
    df = pd.DataFrame({
        'hour': hours,
        'busy_score': x,
        'zone': zone
    })
    df['hour'] = df['hour'].dt.tz_localize(None)
    return df



In [ ]:
# Example usage
start_hour = max(df['hour'])  # specify the start hour
zone = 1  # specify the zone

new_df = create_dataframe(start_hour, zone, x)

In [ ]:
new_df

In [ ]:
zone_list[-1]

In [ ]:
df

In [ ]:
import time
import psycopg2
from io import StringIO

def upload_to_postgres(dataframe, postgres_host, postgres_port, database_name, db_user, db_password, table_name):
    
    start_time = time.time()
    
    conn = psycopg2.connect(
        database=database_name,
        user=db_user,
        password=db_password,
        host=postgres_host,
        port=postgres_port
    )
    print("PostgreSQL connection opened")

    # Example: Execute SQL query
    cursor = conn.cursor()
        
    # Create a temporary table with DOUBLE PRECISION for lat/lng and BIGINT for Unix timestamps
    cursor.execute('''
    CREATE TEMP TABLE temp (
        hour TIMESTAMP WITHOUT TIME ZONE,
        score DOUBLE PRECISION,
        zone INTEGER
    );
    ''')

    # Using StringIO to perform bulk insert into the temporary table
    buffer = StringIO()
    dataframe.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cursor.copy_expert(f"""
    COPY temp (hour, score, zone)
    FROM STDIN WITH CSV
    """, buffer)

    # Insert data from the temporary table to the target table with conflict handling
    cursor.execute(f"""
    INSERT INTO {table_name} (hour, score, zone)
    SELECT hour, score, zone
    FROM temp
    ON CONFLICT DO NOTHING;
    """)

    # Commit the transaction
    try:
        conn.commit()
        print("Data inserted successfully")
    except Exception as e:
        print(f"Unexpected error: {e}")

    # Close the PostgreSQL connection
    cursor.close()
    conn.close()
    print("PostgreSQL connection closed")

    print(f'connect_to_postgres took {time.time() - start_time} seconds to complete')


In [ ]:
postgres_host = 'localhost'
postgres_port = 5432 
database_name = 'busyness'
db_user = 'crafty'
db_password = 'winner'
table_name = 'public.busyness_score'

connect_to_postgres(new_df, postgres_host, postgres_port, database_name, db_user, db_password, table_name)

In [ ]:

connect_to_postgres(new_df, postgres_host, postgres_port, database_name, db_user, db_password)


In [ ]:
query = '''
    SELECT
    *
    FROM
        busyness_score; 
    '''

In [ ]:
df = connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user, db_password)

In [ ]:
df

In [ ]:
df2 = pd.read_csv('/Users/brianmcmahon/Downloads/MHTN_zoned_streets.csv')
df2 = df2.drop(columns = ['address','street_geometry','zone_name','zone_geometry','zone_id'])
df2

In [ ]:
df2 = df2.drop(columns = ['address','street_geometry','zone_name','zone_geometry','zone_id'])

In [ ]:
df2

In [ ]:
taxi_zone = gpd.read_file('NYCTaxiZones.geojson')
taxi_zone.set_geometry("geometry", inplace=True)
taxi_zone['centroid'] = taxi_zone['geometry'].centroid
taxi_zone['location_id'] = taxi_zone['location_id'].astype('int')

taxi_zone = taxi_zone.drop(columns = ['shape_area','objectid','shape_leng','borough','geometry','zone'])
taxi_zone

In [ ]:
def find_closest_rows(df, target_datetime):
    
#     df = df.merge(taxi_zone, left_on='zone', right_on='location_id', how='left')
#     df = df.drop(columns = ['location_id'])


    # Ensure 'hour' is datetime
    df['hour'] = pd.to_datetime(df['hour'])
    
    # Calculate the absolute difference between each 'hour' and the target_datetime
    df['time_diff'] = abs(df['hour'] - target_datetime)
    
    # Sort by zone and then by the time difference
    df_sorted = df.sort_values(by=['zone', 'time_diff'])
    
    # Drop duplicates to keep the row with the smallest time difference for each zone
    closest_rows = df_sorted.drop_duplicates(subset=['zone'], keep='first')
    
    # Drop the temporary 'time_diff' column
    closest_rows = closest_rows.drop(columns=['time_diff'])
    
    return closest_rows

In [ ]:
df

In [ ]:
target_datetime = pd.to_datetime('2024-06-04 02:30:00')

closest_rows = find_closest_rows(df, target_datetime)

In [ ]:
closest_rows

In [ ]:
df2['score'] = sum(closest_rows['score'])*np.array([i for i in range(len(df2['score']))])
df2

In [ ]:
x = [i for i in range(len(closest_rows['score']))]

In [ ]:
closest_rows['score']*x

In [ ]:
closest_rows['centroid']

In [ ]:
def calculate_distance(geometry, closest_rows):
    x = [(geometry.distance(i))**2 for i in closest_rows['centroid']]
    y = closest_rows['score']
    
    return sum(y/x)
#     return geometry.distance(reference_point)

In [ ]:
from shapely import wkt

df2['street_centroid'] = df2['street_centroid'].apply(wkt.loads)

gdf = gpd.GeoDataFrame(df2, geometry='street_centroid')


In [ ]:
# gdf['distance_to_ref'] = 
gdf['estimate'] = gdf['street_centroid'].apply(calculate_distance, closest_rows = closest_rows)/10**5
gdf['estimate']

In [ ]:
gdf

In [ ]:
from shapely import wkb

df = connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user, db_password)
# print(df)
df['centroid'] = df['centroid'].apply(wkb.loads)
df
# gdf = gpd.GeoDataFrame(df, geometry='centroid', crs = 4326)
# gdf

In [ ]:
df2 = pd.read_csv('/Users/brianmcmahon/Downloads/MHTN_zoned_streets.csv')
df2 = df2.drop(columns = ['address','street_geometry','zone_name','zone_geometry','zone_id'])
df2

In [ ]:
target_datetime = pd.to_datetime('2024-06-04 02:30:00')

taxi_zone = gpd.read_file('NYCTaxiZones.geojson')
taxi_zone.set_geometry("geometry", inplace=True)
taxi_zone['centroid'] = taxi_zone['geometry'].centroid
taxi_zone['location_id'] = taxi_zone['location_id'].astype('int')

taxi_zone = taxi_zone.drop(columns = ['shape_area','objectid','shape_leng','borough','geometry','zone'])
taxi_zone

In [ ]:
def estimate_busyness(query, target_datetime, postgres_host, postgres_port, database_name, db_user, db_password):
    start_time = time.time()

    df = connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user, db_password)
    df['centroid'] = df['centroid'].apply(wkb.loads)

    closest_rows = find_closest_rows(df, target_datetime)
    
    df2 = pd.read_csv('/Users/brianmcmahon/Downloads/MHTN_zoned_streets.csv')
    df2 = df2.drop(columns = ['address','zone_name','zone_geometry','zone_id'])#,'street_geometry'
    df2['street_centroid'] = df2['street_centroid'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df2, geometry='street_centroid')
    gdf['estimate'] = gdf['street_centroid'].apply(calculate_distance, closest_rows = closest_rows)/10**5
    print(time.time() - start_time)
    return gdf

In [ ]:
target_datetime = pd.to_datetime('2024-06-10 12:30:00')

estimate = estimate_busyness(query, target_datetime, postgres_host, postgres_port, database_name, db_user, db_password)
estimate

In [ ]:
taxi_zone

In [ ]:
import time
import psycopg2
from io import StringIO
from shapely import wkb,wkt

def upload_to_postgres(dataframe, postgres_host, postgres_port, database_name, db_user, db_password, table_name):
    
    start_time = time.time()
    
    conn = psycopg2.connect(
        database=database_name,
        user=db_user,
        password=db_password,
        host=postgres_host,
        port=postgres_port
    )
    print("PostgreSQL connection opened")

    # Example: Execute SQL query
    cursor = conn.cursor()
        
    # Create a temporary table with DOUBLE PRECISION for lat/lng and BIGINT for Unix timestamps
    cursor.execute('''
    CREATE TEMP TABLE temp (
        zone INTEGER,
        centroid GEOMETRY
    );
    ''')

    # Using StringIO to perform bulk insert into the temporary table
    buffer = StringIO()
    dataframe.to_csv(buffer, index=False, header=False)
    buffer.seek(0)

    cursor.copy_expert(f"""
    COPY temp (zone, centroid)
    FROM STDIN WITH CSV
    """, buffer)

    # Insert data from the temporary table to the target table with conflict handling
    cursor.execute(f"""
    INSERT INTO {table_name} (zone, centroid)
    SELECT zone, centroid
    FROM temp
    ON CONFLICT DO NOTHING;
    """)

    # Commit the transaction
    try:
        conn.commit()
        print("Data inserted successfully")
    except Exception as e:
        print(f"Unexpected error: {e}")

    # Close the PostgreSQL connection
    cursor.close()
    conn.close()
    print("PostgreSQL connection closed")

    print(f'connect_to_postgres took {time.time() - start_time} seconds to complete')


In [ ]:
# upload_to_postgres(taxi_zone, postgres_host, postgres_port, database_name, db_user, db_password, 'taxi_zone')

In [ ]:
import time
import psycopg2
from io import StringIO
from shapely import wkb,wkt
import numpy as np
import pandas as pd 
import geopandas as gpd
import os
from shapely.geometry import Point
from shapely.geometry import MultiPolygon, Polygon


postgres_host = 'localhost'
postgres_port = 5432 
database_name = 'busyness'
db_user = 'crafty'
db_password = 'winner'
table_name = 'public.busyness_data'

query = '''
    SELECT
    *
    FROM
        busyness_score; 
    '''

def connect_to_postgres(query, postgres_host, postgres_port, database_name, db_user , db_password):
    start_time = time.time()
    
    conn = None
    try:
        # Connect to PostgreSQL
        conn = psycopg2.connect(
            database=database_name,
            user=db_user,
            password=db_password,
            host=postgres_host,
            port=postgres_port
        )
        print("PostgreSQL connection opened")
        
        # Create a cursor object
        cursor = conn.cursor()
        
        # Execute the query
        cursor.execute(query)
        print("Query executed successfully")
        
        # Fetch all rows from the executed query
        rows = cursor.fetchall()
        
        # Get column names from the cursor
        colnames = [desc[0] for desc in cursor.description]
        
        # Convert the result to a pandas DataFrame
        df = pd.DataFrame(rows, columns=colnames)
        
        return df

    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

    finally:
        # Closing the connection
        if conn:
            cursor.close()
            conn.close()
            print("PostgreSQL connection closed.")

    print(f'connect_to_postgres took {time.time() - start_time} seconds to complete')
    
def find_closest_rows(df, target_datetime):
    
#     df = df.merge(taxi_zone, left_on='zone', right_on='location_id', how='left')
#     df = df.drop(columns = ['location_id'])


    # Ensure 'hour' is datetime
    df['hour'] = pd.to_datetime(df['hour'])
    
    # Calculate the absolute difference between each 'hour' and the target_datetime
    df['time_diff'] = abs(df['hour'] - target_datetime)
    
    # Sort by zone and then by the time difference
    df_sorted = df.sort_values(by=['zone', 'time_diff'])
    
    # Drop duplicates to keep the row with the smallest time difference for each zone
    closest_rows = df_sorted.drop_duplicates(subset=['zone'], keep='first')
    
    # Drop the temporary 'time_diff' column
    closest_rows = closest_rows.drop(columns=['time_diff'])
    
    return closest_rows



In [ ]:
query = '''
    SELECT
    *
    FROM
        busyness_score; 
    '''
postgres_host = 'localhost'
postgres_port = 5432 
database_name = 'busyness'
db_user = 'crafty'
db_password = 'winner'
table_name = 'public.busyness_data'

In [ ]:
target_datetime = pd.to_datetime('2024-06-05 12:30:00')

estimate = estimate_busyness(query, target_datetime, postgres_host, postgres_port, database_name, db_user, db_password)
estimate_2 = estimate.sort_values(by='estimate', ascending=False)
estimate_2
e3 = estimate_2.head(60)
e3

In [ ]:
target_datetime = pd.to_datetime('2024-06-04 02:30:00')

estimate = estimate_busyness(query, target_datetime, postgres_host, postgres_port, database_name, db_user, db_password)
estimate_2 = estimate.sort_values(by='estimate', ascending=False).head(10)
e3 = estimate_2.head(10)
e3

In [ ]:
import pandas as pd
from geopy.distance import geodesic


def parse_event_datetime(dt_str):
    formats = ["%Y-%m-%dT%H:%M:%S.%f", "%Y-%m-%d %H:%M:%S", "%Y-%m-%dT%H:%M:%S"]
    for fmt in formats:
        try:
            return pd.to_datetime(dt_str, format=fmt)
        except ValueError:
            continue
    raise ValueError(f"Unable to parse datetime string: {dt_str}")

def find_nearest_event(events_df, location_lat, location_lng, user_datetime):
    events_df['Distance'] = events_df.apply(
        lambda row: geodesic((location_lat, location_lng), (row['Latitude'], row['Longitude'])).meters,
        axis=1
    )
    valid_events = events_df[
        (events_df['Start Date/Time'] <= user_datetime) & (events_df['End Date/Time'] >= user_datetime)]
    if valid_events.empty:
        return None
    return valid_events.loc[valid_events['Distance'].idxmin()]

def get_recommendations(busy_score_data, events_df, user_zone, user_datetime):
    events_df['Start Date/Time'] = pd.to_datetime(events_df['Start Date/Time'],format='mixed')
    events_df['End Date/Time'] = pd.to_datetime(events_df['End Date/Time'],format='mixed')
    relevant_areas = busy_score_data

    if relevant_areas.empty:
        return pd.DataFrame()

    recommendations = relevant_areas.apply(
        lambda row: pd.Series({
            'Latitude': row['street_centroid'].y,
            'Longitude': row['street_centroid'].x,
            'Zone': user_zone,
            'Score': row['estimate'],
            'DistanceToEvent': find_nearest_event(events_df, row['street_centroid'].y, row['street_centroid'].x, user_datetime)['Distance'] if find_nearest_event(events_df, row['street_centroid'].y, row['street_centroid'].x, user_datetime) is not None else float('inf')
        }),
        axis=1
    )

    recommendations = recommendations.sort_values(by='Score', ascending=False)

    return recommendations


In [ ]:
events_df['Start Date/Time'] = pd.to_datetime(events_df['Start Date/Time'],format='mixed')
events_df['End Date/Time'] = pd.to_datetime(events_df['End Date/Time'],format='mixed')
events_df


In [ ]:
target_datetime = pd.to_datetime('2024-07-27 17:30:00')

get_recommendations(e3, events_df, 3, target_datetime)

In [ ]:
events_df = pd.read_csv('combined_nyc_events.csv')
find_nearest_event(events_df, events_df, 3, target_datetime)

In [ ]:
events_df['Start Date/Time']

In [ ]:
import pandas as pd

# Replace 'output.csv' with the actual path returned by connect_to_postgres
csv_path = '/Users/brianmcmahon/Documents/Research Practicum/Data/test.csv'

# Load the CSV into a pandas DataFrame
df = pd.read_csv(csv_path)

# Now you can use df for further analysis or processing in your Jupyter Notebook
df

In [ ]:
df['centroid'].str[10:]
